# Setting and running Freyberg with pilot points...

### This should be super easy...

In [1]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu

In [2]:
import freyberg_setup as fs
fs.setup_pest_pp()
working_dir = fs.WORKING_DIR_PP
pst_name = fs.PST_NAME_PP

****Warning -- two packages of the same type:  <class 'flopy.modflow.mfbas.ModflowBas'> <class 'flopy.modflow.mfbas.ModflowBas'>
replacing existing Package...

changing model workspace...
   freyberg_pp
FloPy is using the following executable to run the model: /Users/jeremyw/Dev/gw1876/activities/freyberg_pilot_points_1/freyberg_pp/mfnwt

                                  MODFLOW-NWT-SWR1 
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUNDWATER-FLOW MODEL
                             WITH NEWTON FORMULATION
                             Version 1.1.1, 7/21/2016                        
                    BASED ON MODFLOW-2005 Version 1.11.0 08/08/2013                       

                    SWR1 Version 1.04.0 07/21/2016                       

 Using NAME file: freyberg.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2018/03/21 12:41:36

 Solving:  Stress period:     1    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    

## Let's go explore the files that were created for us.
### somethings to note:
### - what are the parameters that start with ``w``?
### - how many pilot points are there?
### - anything else look different?

In [3]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax

8

In [4]:
pst.pestpp_options

{'forecasts': 'travel_time,fa_headwaters_0001,c001fr16c17_19791231,c001fr05c04_19791231',
 'lambda_scale_fac': '1.0',
 'lambdas': '0.1,1.0,10.0',
 'n_iter_base': '-1',
 'n_iter_super': '3',
 'upgrade_augment': 'false'}

In [5]:
pst.npar

47

### Ok - let's run this bad boy...

In [6]:
os.chdir(working_dir)
pyemu.helpers.start_slaves('.',"pestpp",pst_name,num_slaves=15,master_dir='.')
os.chdir('..')

In [7]:
jco = pyemu.Jco.from_binary(os.path.join(working_dir,pst_name.replace(".pst",".jcb"))).to_dataframe()

In [8]:
jco.loc[pst.nnz_obs_names,:]

,hk00,hk01,hk02,hk03,hk04,hk05,hk06,hk07,hk08,hk09,...,w0_r20_c14,w0_r26_c10,w0_r29_c06,w0_r34_c12,w1_r09_c16,w1_r11_c13,w1_r20_c14,w1_r26_c10,w1_r29_c06,w1_r34_c12
c001cr03c10_19700102,-1.634713e-02,0.253445,-1.919634,-0.932543,0.006276,-0.221759,-0.958834,-0.738931,-0.043246,-0.522845,...,3.613311e-13,3.613311e-13,3.613311e-13,3.613311e-13,3.613311e-13,3.613311e-13,3.613311e-13,3.613311e-13,3.613311e-13,3.613311e-13
c001cr03c16_19700102,-6.964629e-03,0.044508,0.098017,-0.242309,0.001096,0.027010,0.033498,-0.465164,-0.003914,-0.031367,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr04c09_19700102,2.521267e-02,-0.128993,-1.998100,-0.609134,0.057359,-0.643783,-1.897159,-0.716991,-0.072310,-0.923671,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr10c02_19700102,-6.163276e-03,-0.720399,-0.935820,-0.220438,-0.523149,-2.493848,-2.269023,-0.472725,-0.373709,-1.412564,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr14c11_19700102,-8.863236e-03,-0.120442,-0.286171,-0.107746,0.051423,0.302670,-0.444167,-0.402528,0.077568,-0.646576,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr16c17_19700102,1.783470e-03,0.004844,-0.010762,0.034488,0.003513,0.003799,-0.017094,0.047104,0.004835,0.004493,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr22c11_19700102,1.379340e-03,-0.064086,-0.089847,-0.016240,-0.035183,-0.198331,-0.208697,-0.043545,-0.015780,-0.139406,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr23c16_19700102,2.302737e-04,-0.001561,-0.003167,0.005245,-0.000120,-0.002878,-0.007886,0.008638,0.000230,-0.007308,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr25c05_19700102,3.625660e-03,-0.142677,-0.187176,-0.043365,-0.088978,-0.496065,-0.451235,-0.095024,-0.047970,-0.288359,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
c001cr27c07_19700102,3.395386e-03,-0.132082,-0.173536,-0.040589,-0.082407,-0.459091,-0.418022,-0.088338,-0.043476,-0.267523,...,-3.613311e-13,-3.613311e-13,-3.613311e-13,-3.613311e-13,-3.613311e-13,-3.613311e-13,-3.613311e-13,-3.613311e-13,-3.613311e-13,-3.613311e-13


### Let's see how we did:

In [9]:
pst.phi

251.23208218057275

### Wow - we slayed ``phi`` this time

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
df = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".iobj")))
ax = plt.subplot(111)
ax.plot(df.model_runs_completed,df.total_phi,marker='.')
ax.set_xlabel("model runs")
ax.set_ylabel("$\phi$")

In [ ]:
ax = plt.subplot(111,aspect="equal") 
wl_names = [n for n in pst.nnz_obs_names if n.startswith("c")]
ax.scatter(pst.res.loc[wl_names,"modelled"],pst.res.loc[wl_names,"measured"],marker='.')
ylim,xlim = ax.get_ylim(),ax.get_xlim()
mn = min(ylim[0],xlim[0])
mx = max(ylim[1],xlim[1])
ax.plot([mn,mx],[mn,mx])
ax.set_ylim(mn,mx)
ax.set_xlim(mn,mx)

### Sweet!  We nailed the heads!  This must be the perfect model in every way!  But, before we finalize the report, let's check out the parameter values:

In [ ]:
df_paru = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".par.usum.csv")),index_col=0)
hk_pars = [p.upper() for p in pst.par_names if p.startswith("hk")]
df_hk = df_paru.loc[hk_pars,:]
ax = pyemu.helpers.plot_summary_distributions(df_hk,label_post=True)
mn = np.log10(pst.parameter_data.loc[hk_pars[0].lower(),"parlbnd"])
mx = np.log10(pst.parameter_data.loc[hk_pars[0].lower(),"parubnd"])
ax.plot([mn,mn],ax.get_ylim(),"k--")
ax.plot([mx,mx],ax.get_ylim(),"k--")

### Lots of HK pars at their bounds...not a good sign.  We should probably see how the ``HK`` field looks:

In [ ]:
os.chdir(working_dir)
pst.parrep(pst_name.replace(".pst",".parb"))
pyemu.pst_utils.write_parvals_in_tplfiles(pst)
pyemu.gw_utils.fac2real("hkpp.dat",factors_file="hkpp.dat.fac",out_file="hk_layer_1.ref")
os.chdir("..")

In [ ]:
df_pp = pyemu.gw_utils.pp_tpl_to_dataframe(os.path.join(working_dir,"hkpp.dat.tpl"))
m = flopy.modflow.Modflow.load(fs.MODEL_NAM,model_ws=working_dir)
ax = m.upw.hk[0].plot(colorbar=True,alpha=0.5)
ax.scatter(df_pp.x,df_pp.y,marker='x')

In [ ]:
m.upw.hk[0] = os.path.join(fs.BASE_MODEL_DIR,"hk.truth.ref")
ax = m.upw.hk[0].plot(colorbar=True,alpha=0.5)
ax.scatter(df_pp.x,df_pp.y,marker='x')

### Something is wrong...how does the calibrated HK field have so much more variability than the "truth"?  We better checkout the forecasts:

In [ ]:
figs, axes = pyemu.helpers.plot_summary_distributions(os.path.join(working_dir,
                    pst_name.replace(".pst",".pred.usum.csv")),subplots=True)
for ax in axes:
    fname = ax.get_title().lower()
    ylim = ax.get_ylim()
    v = pst.observation_data.loc[fname,"obsval"]
    ax.plot([v,v],ylim,"b--")
    ax.set_ylim(ylim)

### Doh! We are way off for lots of forecasts (compared to the "truth").  So what happened?  Answer: overfitting: we specified lots of parameters, so we are able to fit the observations really well - too well.  

### Even though we are able to measure water levels very precisely, the model has problems (model error), so we shouldn't expect the model to reproduce the observations so well.  But how do we control this overfitting??? 